1. Determine los tres países con mayor número de jugadores(jugadores nacidos en ese país). El resultado debe estar ordenado de forma descendente.

2. Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas, el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.

3. ¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas, el nombre de la liga y la cantidad de juegos que se jugaron en ella.

4. Obtenga las tres ligas con mayor número de asistencia de público teniendo en cuenta todos los juegos que se jugaron en ellas. El resultado debe estar ordenado de forma descendente y tener dos columnas, el nombre de la liga y la asistencia total.

In [1]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import col, desc, sum, sum_distinct, min, max, avg, count, countDistinct, \
    approx_count_distinct, broadcast

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc: SparkContext = spark.sparkContext

In [6]:
appearances = spark.read.csv(
    r"../../data/appearances.csv",
    header=True,
    inferSchema=True,
)
appearances.printSchema()
appearances.show(5)

root
 |-- player_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- appearance_id: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)

+---------+-------+-------------+--------------+--------------+-----+-------+--------------+------------+---------+
|player_id|game_id|appearance_id|competition_id|player_club_id|goals|assists|minutes_played|yellow_cards|red_cards|
+---------+-------+-------------+--------------+--------------+-----+-------+--------------+------------+---------+
|    52453|2483937|2483937_52453|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2479929|2479929_67064|           RU1|         28095|    0|      0|            90|      

In [7]:
clubs = spark.read.csv(
    r"../../data/clubs.csv",
    header=True,
    inferSchema=True,
)
clubs.printSchema()
clubs.show(5)

root
 |-- club_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- pretty_name: string (nullable = true)
 |-- domestic_competition_id: string (nullable = true)
 |-- total_market_value: double (nullable = true)
 |-- squad_size: integer (nullable = true)
 |-- average_age: double (nullable = true)
 |-- foreigners_number: integer (nullable = true)
 |-- foreigners_percentage: double (nullable = true)
 |-- national_team_players: integer (nullable = true)
 |-- stadium_name: string (nullable = true)
 |-- stadium_seats: integer (nullable = true)
 |-- net_transfer_record: string (nullable = true)
 |-- coach_name: string (nullable = true)
 |-- url: string (nullable = true)

+-------+-------------------+-------------------+-----------------------+------------------+----------+-----------+-----------------+---------------------+---------------------+--------------------+-------------+-------------------+----------------+--------------------+
|club_id|               name|        p

In [8]:
competitions = spark.read.csv(
    r"../../data/competitions.csv",
    header=True,
    inferSchema=True,
)
competitions.printSchema()
competitions.show(5)

root
 |-- competition_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- country_id: integer (nullable = true)
 |-- country_name: string (nullable = true)
 |-- domestic_league_code: string (nullable = true)
 |-- confederation: string (nullable = true)
 |-- url: string (nullable = true)

+--------------+---------------+------------------+----------+------------+--------------------+-------------+--------------------+
|competition_id|           name|              type|country_id|country_name|domestic_league_code|confederation|                 url|
+--------------+---------------+------------------+----------+------------+--------------------+-------------+--------------------+
|            L1|     bundesliga|        first_tier|        40|     Germany|                  L1|       europa|https://www.trans...|
|           DFB|      dfb-pokal|      domestic_cup|        40|     Germany|                  L1|       europa|https://www.trans..

In [9]:
games = spark.read.csv(
    r"../../data/games.csv",
    header=True,
    inferSchema=True,
)
games.printSchema()
games.show(5)

root
 |-- game_id: integer (nullable = true)
 |-- competition_code: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- round: string (nullable = true)
 |-- date: string (nullable = true)
 |-- home_club_id: integer (nullable = true)
 |-- away_club_id: integer (nullable = true)
 |-- home_club_goals: integer (nullable = true)
 |-- away_club_goals: integer (nullable = true)
 |-- home_club_position: integer (nullable = true)
 |-- away_club_position: integer (nullable = true)
 |-- stadium: string (nullable = true)
 |-- attendance: integer (nullable = true)
 |-- referee: string (nullable = true)
 |-- url: string (nullable = true)

+-------+----------------+------+-------------+----------+------------+------------+---------------+---------------+------------------+------------------+--------------------+----------+-----------------+--------------------+
|game_id|competition_code|season|        round|      date|home_club_id|away_club_id|home_club_goals|away_club_goals|home_clu

In [10]:
leagues = spark.read.csv(
    r"../../data/leagues.csv",
    header=True,
    inferSchema=True,
)
leagues.printSchema()
leagues.show(5)

root
 |-- league_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- confederation: string (nullable = true)

+---------+------------------+-------------+
|league_id|              name|confederation|
+---------+------------------+-------------+
|       L1|        bundesliga|       europa|
|      NL1|        eredivisie|       europa|
|      BE1|jupiler-pro-league|       europa|
|      ES1|            laliga|       europa|
|      PO1|          liga-nos|       europa|
+---------+------------------+-------------+
only showing top 5 rows



In [5]:
players = spark.read.csv(
    r"../../data/players.csv",
    header=True,
    inferSchema=True,
)
players.printSchema()
players.show(5)

root
 |-- player_id: integer (nullable = true)
 |-- current_club_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- pretty_name: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- position: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: integer (nullable = true)
 |-- market_value_in_gbp: double (nullable = true)
 |-- highest_market_value_in_gbp: double (nullable = true)
 |-- url: string (nullable = true)

+---------+---------------+-------------------+-------------------+----------------+----------------------+-------------+----------+--------------+-----+------------+-------------------+---------------------------+--------------------+
|player_id|current_club_id|               name|        pretty_name|country_of_birth|country_of_citizenship|date_of_birth|  position|

In [20]:
# 1. Determine los tres países con mayor número de jugadores(jugadores nacidos en ese país). El resultado debe estar ordenado de forma descendente.

(
    players
    .dropna(
        subset=["country_of_birth"],
    )
    .groupBy("country_of_birth")
    .count()
    .orderBy(desc("count"))
    .show(3)
)

+----------------+-----+
|country_of_birth|count|
+----------------+-----+
|          France| 1694|
|           Spain| 1388|
|           Italy| 1312|
+----------------+-----+
only showing top 3 rows



In [23]:
# 2. Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas, el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.

print(players.columns)
print(appearances.columns)

['player_id', 'current_club_id', 'name', 'pretty_name', 'country_of_birth', 'country_of_citizenship', 'date_of_birth', 'position', 'sub_position', 'foot', 'height_in_cm', 'market_value_in_gbp', 'highest_market_value_in_gbp', 'url']
['player_id', 'game_id', 'appearance_id', 'competition_id', 'player_club_id', 'goals', 'assists', 'minutes_played', 'yellow_cards', 'red_cards']


In [38]:
players_cards = (
    players.join(
        appearances,
        on="player_id",
        how="inner",
    )
)

(
    players_cards
    .groupBy(
        "player_id",
        "pretty_name",
    )
    .agg(
        sum(
            "red_cards"
        ).alias("red_cards")
    )
    .orderBy(
        desc("red_cards")
    )
    .select(
        "pretty_name",
        "red_cards",
    )
    .show()
)

+--------------------+---------+
|         pretty_name|red_cards|
+--------------------+---------+
|        Granit Xhaka|        5|
|    Domenico Berardi|        5|
|              Hilton|        5|
|          Thomas Lam|        5|
|     Mario Balotelli|        5|
|     Damien Da Silva|        4|
|     Rodrigo De Paul|        4|
|             Rodrigo|        4|
|         Ivan Ordets|        4|
|   Kalidou Koulibaly|        4|
|          Jordi Amat|        4|
|Timothee Kolodzie...|        4|
|          Allan Nyom|        4|
|     Roberto Soriano|        4|
|       Heung Min Son|        4|
|       Ramon Leeuwin|        4|
|      Ognjen Vranjes|        4|
|     Gabriel Paletta|        4|
|         Jaume Costa|        4|
|     Yannick Cahuzac|        4|
+--------------------+---------+
only showing top 20 rows



In [48]:
# 3. ¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas, el nombre de la liga y la cantidad de juegos que se jugaron en ella.
print(competitions.columns)
print(games.columns)

['competition_id', 'name', 'type', 'country_id', 'country_name', 'domestic_league_code', 'confederation', 'url']
['game_id', 'competition_code', 'season', 'round', 'date', 'home_club_id', 'away_club_id', 'home_club_goals', 'away_club_goals', 'home_club_position', 'away_club_position', 'stadium', 'attendance', 'referee', 'url']
+--------------+--------------+----------+----------+------------+--------------------+-------------+--------------------+
|competition_id|          name|      type|country_id|country_name|domestic_league_code|confederation|                 url|
+--------------+--------------+----------+----------+------------+--------------------+-------------+--------------------+
|           GB1|premier-league|first_tier|       189|     England|                 GB1|       europa|https://www.trans...|
+--------------+--------------+----------+----------+------------+--------------------+-------------+--------------------+



In [84]:
games_premier = (
    competitions.join(
        other=games,
        on=competitions.competition_id == games.competition_code,
        how="inner",
    )
    .groupBy(
        "competition_id",
        "name",
    )
    .agg(
        count("*").alias("games_count")
    )
    .select("name", "games_count")
    .where(col("name") == "premier-league")
)

games_premier.show()

+--------------+-----------+
|          name|games_count|
+--------------+-----------+
|premier-league|       2809|
+--------------+-----------+



In [75]:
# 4. Obtenga las tres ligas con mayor número de asistencia de público teniendo en cuenta todos los juegos que se jugaron en ellas. El resultado debe estar ordenado de forma descendente y tener dos columnas, el nombre de la liga y la asistencia total.

print(leagues.columns)
print(competitions.columns)
print(games.columns)
# print(clubs.columns)

['league_id', 'name', 'confederation']
['competition_id', 'name', 'type', 'country_id', 'country_name', 'domestic_league_code', 'confederation', 'url']
['game_id', 'competition_code', 'season', 'round', 'date', 'home_club_id', 'away_club_id', 'home_club_goals', 'away_club_goals', 'home_club_position', 'away_club_position', 'stadium', 'attendance', 'referee', 'url']


In [82]:
competitions_leagues = (
    competitions.join(
        other=leagues,
        on=competitions.competition_id == leagues.league_id,
        how="inner",
    )
)

leagues_games = (
    competitions_leagues.join(
        other=games,
        on=competitions_leagues.competition_id == games.competition_code,
        how="inner",
    )
)

(
    leagues_games
    .groupBy(
        competitions.competition_id,
        competitions.name
    )
    .agg(
        sum(games.attendance).alias("total_attendance")
    )
    .orderBy(
        desc("total_attendance")
    )
    .select(
        competitions.name,
        "total_attendance",
    )
    .show()
)

+--------------------+----------------+
|                name|total_attendance|
+--------------------+----------------+
|      premier-league|        86964852|
|          bundesliga|        78102473|
|              laliga|        62943533|
|             serie-a|        53475147|
|             ligue-1|        51593963|
|          eredivisie|        34572418|
|        premier-liga|        20878744|
|  liga-portugal-bwin|        20072843|
|  jupiler-pro-league|        17817099|
|           super-lig|        17455236|
|scottish-premiership|        17379753|
|         superligaen|         7945555|
|      super-league-1|         6417136|
|        premier-liga|         4944837|
+--------------------+----------------+

